Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

## Answer

## Logistic model with/without regularisation

Find below the code, uncomment "Without Regularisation" to compare results
86.4% vs 88.7% with regularisation ... pretty cool

In [ ]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases

  # Without Regularisation
  #loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  # With Regularization    
  regularization = tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))  + 0.001 * regularization
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [ ]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

## Neural Network model with/without regularisation

Find below the code, uncomment "L2 regularization" to compare results
89.1% vs 93.0% with regularisation ... pretty cool

In [33]:
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_1 = tf.Variable(tf.zeros([hidden_nodes]))
  
  # Training computation.
  #logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  #tf.nn.relu(logits_1)
  layer_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)

  weights_2 = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  layer_2 = tf.matmul(layer_1, weights_2) + biases_2

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(layer_2, tf_train_labels))

  # L2 regularization
  regularizers = (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(biases_1) +
                  tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(biases_2))
  # Add the regularization term to the loss.
  loss += 0.001 * regularizers
  #loss += 5e-4 * regularizers

  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(layer_2)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2
  )
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2) 

In [34]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 722.172424
Minibatch accuracy: 9.4%
Validation accuracy: 30.2%
Minibatch loss at step 500: 198.655945
Minibatch accuracy: 81.2%
Validation accuracy: 78.5%
Minibatch loss at step 1000: 117.510368
Minibatch accuracy: 80.5%
Validation accuracy: 81.7%
Minibatch loss at step 1500: 69.386948
Minibatch accuracy: 89.1%
Validation accuracy: 82.9%
Minibatch loss at step 2000: 41.523720
Minibatch accuracy: 92.2%
Validation accuracy: 84.5%
Minibatch loss at step 2500: 25.399752
Minibatch accuracy: 89.1%
Validation accuracy: 85.7%
Minibatch loss at step 3000: 15.655616
Minibatch accuracy: 84.4%
Validation accuracy: 86.6%
Test accuracy: 93.2%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

## Answer
#### Neural Network model with out regularisation and few batches

So this results in extreme overfitting :O
training accuracy is 100% but validation and test accuracies are quite low.

In [ ]:
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_1 = tf.Variable(tf.zeros([hidden_nodes]))
  
  # Training computation.
  #logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  #tf.nn.relu(logits_1)
  layer_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)

  weights_2 = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  layer_2 = tf.matmul(layer_1, weights_2) + biases_2

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(layer_2, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(layer_2)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2
  )
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2) 

In [ ]:
num_steps = 500

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
        
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    
    # Pick only 3 batches from training data.
    my_step = step%3
    
    offset = (my_step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

## Answer
#### This is Problem 2 but with dropouts
wow that's cool, no overfitting! and results generalise well.

In [ ]:
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  tf_dropout = tf.placeholder(tf.float32)
  
  # Variables.
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_1 = tf.Variable(tf.zeros([hidden_nodes]))
    
  weights_2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  #layer_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
  layer_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  relu_layer = tf.nn.relu(layer_1)
  dropout_layer = tf.nn.dropout(relu_layer, tf_dropout)
  layer_2 = tf.matmul(dropout_layer, weights_2) + biases_2

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(layer_2, tf_train_labels))

  # L2 regularization
  #regularizers = (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(biases_1) +
  #                tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(biases_2))
  # Add the regularization term to the loss.
  #loss += 0.001 * regularizers
  #loss += 5e-4 * regularizers
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(layer_2)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2
  )
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2) 

In [ ]:
num_steps = 500

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    
    # Pick only 3 batches from training data.
    my_step = step%3
    
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, tf_dropout : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


# Some tinkering with dropout

#### With Regularisation only, no dropout
Results: 93.1% accuracy

In [ ]:
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  #tf_dropout = tf.placeholder(tf.float32)
  
  # Variables.
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_1 = tf.Variable(tf.zeros([hidden_nodes]))
  
  weights_2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  layer_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  relu_layer = tf.nn.relu(layer_1)
  #dropout_layer = tf.nn.dropout(relu_layer, tf_dropout)
  #layer_2 = tf.matmul(dropout_layer, weights_2) + biases_2
  layer_2 = tf.matmul(relu_layer, weights_2) + biases_2

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(layer_2, tf_train_labels))

  # L2 regularization
  regularizers = (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(biases_1) + tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(biases_2))
  loss += 0.001 * regularizers
  #loss += 5e-4 * regularizers

  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(layer_2)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2
  )
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2) 

In [ ]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

#### With Regularisation and dropout

hmmm the accuracy is still lower with dropout, even with 4001 steps. we need to increase the depth of network and probably not use dropout.

Note: If I understand correctly dropout are useful if there's a problem of overfitting i.e. less training error more test error

In [ ]:
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  tf_dropout = tf.placeholder(tf.float32)
  
  # Variables.
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_1 = tf.Variable(tf.zeros([hidden_nodes]))
  
  weights_2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  layer_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  relu_layer = tf.nn.relu(layer_1)
  dropout_layer = tf.nn.dropout(relu_layer, tf_dropout)
  layer_2 = tf.matmul(dropout_layer, weights_2) + biases_2
  
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(layer_2, tf_train_labels))

  # L2 regularization
  regularizers = (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(biases_1) + tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(biases_2))
  loss += 0.001 * regularizers
  #loss += 5e-4 * regularizers

  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(layer_2)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2
  )
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2) 

In [ ]:
num_steps = 4001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,tf_dropout : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

## FINAL

In [57]:
batch_size = 128
hidden_nodes_1 = 1024
hidden_nodes_2 = 200

graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  tf_dropout_1 = tf.placeholder(tf.float32)
  tf_dropout_2 = tf.placeholder(tf.float32)

  # Variables.
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes_1]))
  biases_1 = tf.Variable(tf.zeros([hidden_nodes_1]))
  
  weights_2 = tf.Variable(tf.truncated_normal([hidden_nodes_1, hidden_nodes_2]))
  biases_2 = tf.Variable(tf.zeros([hidden_nodes_2]))
    
  weights_3 = tf.Variable(tf.truncated_normal([hidden_nodes_2, num_labels]))
  biases_3 = tf.Variable(tf.zeros([num_labels]))
  
  layer_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  relu_layer_1 = tf.nn.relu(layer_1)
  dropout_layer_1 = tf.nn.dropout(relu_layer_1, tf_dropout_1)
    
  layer_2 = tf.matmul(dropout_layer_1, weights_2) + biases_2
  relu_layer_2 = tf.nn.relu(layer_2)
  dropout_layer_2 = tf.nn.dropout(relu_layer_2, tf_dropout_2)

  layer_3 = tf.matmul(dropout_layer_2, weights_3) + biases_3

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(layer_3, tf_train_labels))

  # L2 regularization
  #regularizers = (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(biases_1)
  #                + tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(biases_2)
  #               + tf.nn.l2_loss(weights_3) + tf.nn.l2_loss(biases_3))
  #loss += 0.001 * regularizers
  
  # Optimizer. 
  optimizer = tf.train.GradientDescentOptimizer(0.0007).minimize(loss)

  #global_step = tf.Variable(0)  # count the number of steps taken.
  #learning_rate = tf.train.exponential_decay(0.2, global_step,1000,0.5)
  #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)


  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(layer_3)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2),weights_3) + biases_3)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2),weights_3) + biases_3)

  #test_prediction = tf.nn.softmax(
  #  tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2
  #) 
#tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2), weights_3) + biases_3

In [58]:
num_steps = 4000

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    
    #if((step%10 == 0) and step >1):
    #    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, tf_dropout_1 : 0.99, tf_dropout_2 : 0.99}
    #else:
        #dropout value is "keep probability"
    #    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, tf_dropout_1 : 1.0, tf_dropout_2 : 1.0}
    
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, tf_dropout_1 : 1.0, tf_dropout_2 : 1.0}
        
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4343.932129
Minibatch accuracy: 10.2%
Validation accuracy: 11.5%
Minibatch loss at step 500: 140.862610
Minibatch accuracy: 82.0%
Validation accuracy: 73.2%
Minibatch loss at step 1000: 178.696915
Minibatch accuracy: 72.7%
Validation accuracy: 75.0%
Minibatch loss at step 1500: 71.835327
Minibatch accuracy: 78.1%
Validation accuracy: 75.5%
Minibatch loss at step 2000: 75.105881
Minibatch accuracy: 79.7%
Validation accuracy: 76.2%
Minibatch loss at step 2500: 72.086037
Minibatch accuracy: 79.7%
Validation accuracy: 75.9%
Minibatch loss at step 3000: 79.163239
Minibatch accuracy: 71.1%
Validation accuracy: 76.2%
Minibatch loss at step 3500: 61.225586
Minibatch accuracy: 82.0%
Validation accuracy: 76.2%
Test accuracy: 83.4%


## Single Layer fully connected
steps: 4001, learning_rate: 1.0, decay_step: 2000, decay%: 0.5 Accuracy 94.7% <br>
steps: 4001, learning_rate: 1.1, decay_step: 2000, decay%: 0.5 Accuracy 94.7% <br>
steps: 4001, learning_rate: 1.1, decay_step: 2100, decay%: 0.5 Accuracy 94.7% <br>
steps: 4001, learning_rate: 1.2, decay_step: 2100, decay%: 0.5 Accuracy 94.4% <br>
steps: 4001, learning_rate: 1.2, decay_step: 2300, decay%: 0.5 Accuracy 94.5% <br>
steps: 4001, learning_rate: 1.2, decay_step: 2300, decay%: 0.7 Accuracy 94.0% <br>

## 2 layers, 1024, 200
0.0008, 4000 steps, no-drop, no-reg. Accuracy 84.4%



In [11]:
1%20

1

In [31]:
tf.truncated_normal?